In [1]:
from model.pointpillars import PointPillars

In [2]:
import torch
from torch import nn
from model.pointpillars import PointPillars
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Assuming `model` is the PointPillars model
pretrained_model_path = r'C:\Users\hussa\OneDrive\Desktop\Projects\ROS2-Modular-Framework-for-End-to-End-Autonomous-Vehicle-Control-from-Raw-Sensor-Data\poinpillar\pre_trained\epoch_160.pth'
model_state_dict = torch.load(pretrained_model_path, map_location=torch.device('cpu'))

# Initialize the PointPillars model
model = PointPillars(
    nclasses=3,
    voxel_size=[0.16, 0.16, 4],
    point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
    max_num_points=32,
    max_voxels=(16000, 40000)  # Training and inference max_voxels
)

# Load the filtered state dict into the model (without the head weights)
model.load_state_dict(model_state_dict, strict=False)

# # Now replace the head layer with Identity

model.neck = nn.Identity()
model.head = nn.Identity()

# Set the model to evaluation mode
model.eval()


PointPillars(
  (pillar_layer): PillarLayer(
    (voxel_layer): Voxelization(
      (voxelizer): HardVoxelization()
    )
  )
  (pillar_encoder): PillarEncoder(
    (conv): Conv1d(9, 64, kernel_size=(1,), stride=(1,), bias=False)
    (bn): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (backbone): Backbone(
    (multi_blocks): ModuleList(
      (0): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (7): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, tra

In [4]:
# Function to load and preprocess PCD file
def load_pcd_file(pcd_path):
    # Load the point cloud data using Open3D
    pcd = o3d.io.read_point_cloud(pcd_path)

    # Convert to numpy array
    pcd_points = np.asarray(pcd.points)  # Shape: (N, 3), where N is the number of points

    # For simplicity, assuming only x, y, z (ignoring intensity, RGB, etc.)
    return pcd_points

# Preprocess the point cloud data for the model
def preprocess_point_cloud(pcd_points):
    # Assuming the model requires specific preprocessing like voxelization
    # Example: Downsample using voxel grid (this is one of many possible approaches)

    # For example, voxel size (you may need to tune this)
    voxel_size = 0.2
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_points)
    downsampled_pcd = pcd.voxel_down_sample(voxel_size)

    # Convert the downsampled points to numpy array
    downsampled_points = np.asarray(downsampled_pcd.points)  # Shape: (N', 3)

    # Convert to torch tensor and add batch dimension
    input_tensor = torch.tensor(downsampled_points, dtype=torch.float32).unsqueeze(0)

    return input_tensor


In [5]:
# Inference function
def run_inference(input_tensor):
    with torch.no_grad():
        # Run the model inference (assuming the model returns features directly)
        features = model(input_tensor)

    return features

In [6]:
# Example usage
pcd_path = r"C:\Users\hussa\OneDrive\Desktop\Projects\Project\MOT\INFRA-3DRC_scene-15\INFRA-3DRC_scene-15\lidar_01\lidar_01__data\lidar_01__2023-06-02-21-28-09-769.pcd"
pcd_points = load_pcd_file(pcd_path)

# Preprocess the point cloud data
input_tensor = preprocess_point_cloud(pcd_points)
print(input_tensor.shape)

# Run inference
features = run_inference(input_tensor)

# Output the features (can be used for further processing)
print("Extracted Features:", features)

torch.Size([1, 25723, 3])
Extracted Features: [tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.1369, 0.0000, 0.2794],
          [0.0000, 0.0000, 0.0000,  ..., 0.2629, 0.0250, 0.1899],
          [0.0000, 0.0000, 0.0456,  ..., 0.5328, 0.1713, 0.1686],
          ...,
          [0.0000, 0.0000, 0.0798,  ..., 0.4787, 0.2366, 0.1198],
          [0.0000, 0.0000, 0.0000,  ..., 0.0627, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.7995, 0.0000, 0.5594,  ..., 0.5028, 0.4703, 0.2044],
          [0.6727, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.6281, 0.0000, 0.0000,  ..., 0.0020, 0.5184, 0.0836],
          ...,
          [0.5007, 0.0000, 0.0000,  ..., 0.0000, 0.5135, 0.0519],
          [1.0141, 0.0000, 0.0365,  ..., 0.2998, 0.7717, 0.2561],
          [0.9833, 0.2227, 0.7294,  ..., 0.8063, 0.8000, 0.1598]],

         [[0.2044, 0.3150, 0.5370,  ..., 0.5015, 0.7708, 0.4275],
          [0.0000, 0.0000, 0.0000,  ..., 0.0815, 0.3351, 0.32